# MaxEnt Distribution Modeling

This notebook implements species distribution modeling using MaxEnt and projects distributions to past time periods.

## Steps:
1. Prepare occurrence data for MaxEnt
2. Prepare climate layers for MaxEnt
3. Run MaxEnt via command line
4. Project to past time periods
5. Generate distribution maps


In [1]:
import yaml
from pathlib import Path

# Configuration with absolute path
CONFIG_PATH = Path("/Users/divyadhole/Capstone-project/config.yaml")
with open(CONFIG_PATH, 'r') as f:
    config = yaml.safe_load(f)

# Set up paths
PROJECT_ROOT = Path("/Users/divyadhole/Capstone-project")
DATA_DIR = PROJECT_ROOT / config['data_dir']
OUTPUT_DIR = PROJECT_ROOT / config['output_dir']
MAXENT_DIR = PROJECT_ROOT / config['maxent_dir']

# Create directories if they don't exist
for directory in [DATA_DIR, OUTPUT_DIR, MAXENT_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

print(f"✅ Configuration loaded successfully")
print(f"Project root: {PROJECT_ROOT}")
print(f"Data directory: {DATA_DIR}")
print(f"Output directory: {OUTPUT_DIR}")

✅ Configuration loaded successfully
Project root: /Users/divyadhole/Capstone-project
Data directory: /Users/divyadhole/Capstone-project/data
Output directory: /Users/divyadhole/Capstone-project/outputs


In [2]:
from pathlib import Path

manual_climate_dir = Path("/Users/divyadhole/Capstone-project/data/paleoclim/LH_v1_2_5m")
print("Directory exists:", manual_climate_dir.exists())

for pattern in ("bio1*.tif", "bio_1*.tif", "bio12*.tif", "bio_12*.tif"):
    matches = list(manual_climate_dir.glob(pattern))
    print(pattern, "→", len(matches), "matches")
    if matches:
        print("  sample:", matches[0].name)

Directory exists: True
bio1*.tif → 0 matches
bio_1*.tif → 11 matches
  sample: bio_18.tif
bio12*.tif → 0 matches
bio_12*.tif → 1 matches
  sample: bio_12.tif


In [3]:
import shutil
from pathlib import Path

manual_climate_dir = Path("/Users/divyadhole/Capstone-project/data/paleoclim/LH_v1_2_5m")
MAXENT_LAYERS = OUTPUT_DIR / "maxent_layers"
MAXENT_LAYERS.mkdir(exist_ok=True, parents=True)

layer_files = []
for tif in manual_climate_dir.glob("bio_*.tif"):
    dest = MAXENT_LAYERS / f"{manual_climate_dir.name}_{tif.name}"
    shutil.copy(tif, dest)
    layer_files.append(dest)

print(f"Copied {len(layer_files)} climate layers for MaxEnt.")

Copied 19 climate layers for MaxEnt.


## Step 1: Prepare occurrence data for MaxEnt


In [4]:
import yaml
from pathlib import Path

# Load configuration
CONFIG_PATH = Path("/Users/divyadhole/Capstone-project/config.yaml")
with open(CONFIG_PATH, 'r') as f:
    config = yaml.safe_load(f)

# Set up paths
PROJECT_ROOT = Path("/Users/divyadhole/Capstone-project")
DATA_DIR = PROJECT_ROOT / config['data_dir']
OUTPUT_DIR = PROJECT_ROOT / config['output_dir']
MAXENT_DIR = PROJECT_ROOT / config['maxent_dir']

# MaxEnt specific configuration
MAXENT_CONFIG = {
    "current_climate_dirs": [
        PROJECT_ROOT / "data/paleoclim/LH_v1_2_5m"  # Last Glacial Maximum
    ],
    "projection_climate_dirs": [
        PROJECT_ROOT / "data/paleoclim/EH_v1_2_5m",  # Early Holocene
        PROJECT_ROOT / "data/paleoclim/LGM_v1_2_5m"  # Last Glacial Maximum
    ],
    "variables": ["bio1", "bio12"],  # Temperature and Precipitation
    "maxent_jar": PROJECT_ROOT / "maxent/maxent.jar",
    "output_dir": OUTPUT_DIR / "maxent_outputs"
}

def gather_current_climate_dirs():
    """Gather current climate directories from config."""
    configured = MAXENT_CONFIG.get("current_climate_dirs", [])
    resolved = []
    for path in configured:
        candidate_path = Path(path)
        if not candidate_path.is_absolute():
            candidate_path = PROJECT_ROOT / candidate_path
        if candidate_path.exists():
            resolved.append(str(candidate_path))
    return resolved

# Create required directories
for directory in [DATA_DIR, OUTPUT_DIR, MAXENT_DIR, MAXENT_CONFIG["output_dir"]]:
    directory.mkdir(parents=True, exist_ok=True)

# Now you can safely call
current_climate_paths = gather_current_climate_dirs()
print("Current climate directories:", current_climate_paths)

Current climate directories: ['/Users/divyadhole/Capstone-project/data/paleoclim/LH_v1_2_5m']


In [5]:
import yaml
from pathlib import Path
import shutil

# Load main configuration
CONFIG_PATH = Path("/Users/divyadhole/Capstone-project/config.yaml")
with open(CONFIG_PATH, 'r') as f:
    config = yaml.safe_load(f)

# Set up base paths
PROJECT_ROOT = Path("/Users/divyadhole/Capstone-project")
OUTPUT_DIR = PROJECT_ROOT / config['output_dir']
PALEO_BASE_DIR = PROJECT_ROOT / "data/paleoclim"

# MaxEnt Configuration
MAXENT_CONFIG = {
    "current_climate_dirs": [
        PALEO_BASE_DIR / "LH_v1_2_5m"  # Current climate
    ],
    "projection_climate_dirs": [
        PALEO_BASE_DIR / "EH_v1_2_5m",  # Early Holocene
        PALEO_BASE_DIR / "LGM_v1_2_5m"  # Last Glacial Maximum
    ],
    "variables": ["bio1", "bio12"],  # Temperature and Precipitation
    "output_dir": OUTPUT_DIR / "maxent_outputs"
}

# CLIMATE_CONFIG for backward compatibility
CLIMATE_CONFIG = {
    "current": PALEO_BASE_DIR / "LH_v1_2_5m",
    "paleo": PALEO_BASE_DIR,
    "variables": MAXENT_CONFIG["variables"]
}

def _variant_names(var):
    """Generate possible filename variants for a climate variable."""
    names = {var}
    if var.startswith("bio"):
        suffix = var[3:]
        if suffix.startswith("_"):
            suffix = suffix[1:]
        names.update({f"bio{suffix}", f"bio_{suffix}"})
    return list(names)

def prepare_maxent_layers(climate_dirs, output_dir, variables, allow_all_bio=False):
    """Copy climate layers (GeoTIFF/ASCII) for MaxEnt."""
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True, parents=True)

    layer_files = []
    for climate_dir in climate_dirs:
        climate_dir = Path(climate_dir)
        if not climate_dir.exists():
            print(f"Skipping missing climate directory: {climate_dir}")
            continue

        for var in variables:
            candidate_names = ["bio_"] if allow_all_bio else _variant_names(var)
            var_files = []
            for name in candidate_names:
                if name == "bio_":
                    var_files.extend(climate_dir.glob("bio_*.tif"))
                    var_files.extend(climate_dir.glob("bio_*.asc"))
                else:
                    var_files.extend(climate_dir.glob(f"{name}.tif"))
                    var_files.extend(climate_dir.glob(f"{name}.asc"))

            if not var_files:
                print(f"  Missing layers for {var} in {climate_dir}")
                continue

            for var_file in var_files:
                dest_file = output_dir / f"{climate_dir.name}_{var_file.name}"
                shutil.copy(var_file, dest_file)
                layer_files.append(dest_file)

    if not layer_files:
        raise FileNotFoundError("No climate layers were prepared. Check climate directories and variable names.")

    print(f"Prepared {len(layer_files)} climate layers for MaxEnt → {output_dir}")
    return layer_files

def gather_current_climate_dirs():
    """Gather current climate directories with fallbacks."""
    configured = MAXENT_CONFIG.get("current_climate_dirs", [])
    if isinstance(configured, (str, Path)):
        configured = [configured]

    fallback_candidates = []
    if CLIMATE_CONFIG.get("current"):
        fallback_candidates.append(CLIMATE_CONFIG["current"])
    
    # Add common subdirectories to check
    fallback_candidates.extend(PALEO_BASE_DIR / name for name in ("current", "present", "LH_v1_2_5m"))

    resolved = []
    for candidate in configured + fallback_candidates:
        if not candidate:
            continue
        candidate_path = Path(candidate).expanduser()
        if candidate_path.exists() and candidate_path not in resolved:
            resolved.append(candidate_path)
    return resolved

# Create required directories
MAXENT_LAYERS = OUTPUT_DIR / "maxent_layers"
MAXENT_LAYERS.mkdir(exist_ok=True, parents=True)

# Get climate data paths
current_climate_paths = gather_current_climate_dirs()
variables = MAXENT_CONFIG.get('variables', ['bio1', 'bio12'])

# Process climate layers
if current_climate_paths:
    try:
        layer_files = prepare_maxent_layers(current_climate_paths, MAXENT_LAYERS, variables)
        print(f"Successfully prepared {len(layer_files)} layers in {MAXENT_LAYERS}")
    except FileNotFoundError as exc:
        print(exc)
        print("Falling back to copying all bio_* layers...")
        try:
            layer_files = prepare_maxent_layers(
                current_climate_paths, 
                MAXENT_LAYERS, 
                variables, 
                allow_all_bio=True
            )
            print(f"Successfully prepared {len(layer_files)} layers in {MAXENT_LAYERS}")
        except Exception as exc2:
            print("Failed to prepare layers:", exc2)
else:
    print(f"No climate directories found. Please check your configuration in {CONFIG_PATH}")
    print("Expected climate data in:", PALEO_BASE_DIR / "LH_v1_2_5m/")

Prepared 2 climate layers for MaxEnt → /Users/divyadhole/Capstone-project/outputs/maxent_layers
Successfully prepared 2 layers in /Users/divyadhole/Capstone-project/outputs/maxent_layers


## Step 4: Project to past time periods


In [6]:
# Step 1 – Prepare occurrence data for MaxEnt
def prepare_maxent_occurrences(occurrence_file, output_file, species_name=None):
    """Format occurrence data for MaxEnt (CSV: species, longitude, latitude)."""
    occurrence_file = Path(occurrence_file)
    output_file = Path(output_file)

    if not occurrence_file.exists():
        raise FileNotFoundError(f"Occurrence file not found: {occurrence_file}")

    df = pd.read_csv(occurrence_file)
    if df.empty:
        raise ValueError("Occurrence table is empty; rerun Notebook 02 or provide valid points.")

    lon_cols = [c for c in df.columns if c.lower() in {"lon", "longitude", "decimallongitude"}]
    lat_cols = [c for c in df.columns if c.lower() in {"lat", "latitude", "decimallatitude"}]
    if not lon_cols or not lat_cols:
        raise ValueError("Occurrence data must contain longitude/latitude columns.")

    lon_col = lon_cols[0]
    lat_col = lat_cols[0]

    if "species" not in df.columns:
        if "scientific_name" in df.columns:
            df["species"] = df["scientific_name"].fillna(species_name or "target_species")
        elif {"GENUS", "SPECIES"}.issubset(df.columns):
            df["species"] = df["GENUS"].str.strip() + " " + df["SPECIES"].str.strip()
        else:
            df["species"] = species_name or "target_species"

    maxent_df = (
        df[["species", lon_col, lat_col]]
        .rename(columns={lon_col: "longitude", lat_col: "latitude"})
        .dropna(subset=["longitude", "latitude"])
        .drop_duplicates()
    )
    if maxent_df.empty:
        raise ValueError("No valid occurrence rows after filtering missing coordinates.")

    maxent_df.to_csv(output_file, index=False, header=False)
    print(f"Prepared {len(maxent_df)} occurrence points for MaxEnt → {output_file}")
    return maxent_df

occurrence_file = Path(MAXENT_CONFIG.get("occurrence_file", OUTPUT_DIR / "occurrence_points_gbif.csv")).expanduser()
maxent_occurrences = OUTPUT_DIR / "maxent_occurrences.csv"
species_name = MAXENT_CONFIG.get("species_name")

try:
    maxent_presence = prepare_maxent_occurrences(occurrence_file, maxent_occurrences, species_name)
except (FileNotFoundError, ValueError) as exc:
    maxent_presence = None
    print(exc)

NameError: name 'pd' is not defined

In [7]:
# Step 2 – Prepare climate layers for MaxEnt
def _variant_names(var):
    names = {var}
    if var.startswith("bio"):
        suffix = var[3:]
        if suffix.startswith("_"):
            suffix = suffix[1:]
        names.update({f"bio{suffix}", f"bio_{suffix}"})
    return list(names)

def prepare_maxent_layers(climate_dirs, output_dir, variables, allow_all_bio=False):
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True, parents=True)
    layer_files = []

    for climate_dir in map(Path, climate_dirs):
        if not climate_dir.exists():
            print(f"Skipping missing climate directory: {climate_dir}")
            continue

        for var in variables:
            candidate_names = ["bio_"] if allow_all_bio else _variant_names(var)
            var_files = []
            for name in candidate_names:
                if name == "bio_":
                    var_files.extend(climate_dir.glob("bio_*.tif"))
                    var_files.extend(climate_dir.glob("bio_*.asc"))
                else:
                    var_files.extend(climate_dir.glob(f"{name}.tif"))
                    var_files.extend(climate_dir.glob(f"{name}.asc"))

            if not var_files:
                print(f"  Missing layers for {var} in {climate_dir}")
                continue

            for var_file in var_files:
                dest_file = output_dir / f"{climate_dir.name}_{var_file.name}"
                shutil.copy(var_file, dest_file)
                layer_files.append(dest_file)

    if not layer_files:
        raise FileNotFoundError("No climate layers were prepared. Check climate directories and variable names.")

    print(f"Prepared {len(layer_files)} climate layers for MaxEnt → {output_dir}")
    return layer_files

def gather_current_climate_dirs():
    configured = MAXENT_CONFIG.get("current_climate_dirs")
    if isinstance(configured, (str, Path)):
        configured = [configured]
    configured = configured or []

    fallback = []
    if CLIMATE_CONFIG.get("current"):
        fallback.append(CLIMATE_CONFIG["current"])
    fallback.extend(PALEO_BASE_DIR / name for name in ("current", "present", "LH_v1_2_5m"))

    resolved = []
    for candidate in configured + fallback:
        if not candidate:
            continue
        candidate_path = Path(candidate).expanduser()
        if candidate_path.exists() and candidate_path not in resolved:
            resolved.append(candidate_path)
    return resolved

current_climate_paths = gather_current_climate_dirs()
MAXENT_LAYERS = OUTPUT_DIR / "maxent_layers"
variables = CLIMATE_CONFIG.get("variables", ["bio1", "bio12"])

if current_climate_paths:
    try:
        layer_files = prepare_maxent_layers(current_climate_paths, MAXENT_LAYERS, variables)
    except FileNotFoundError as exc:
        print(exc)
        print("Falling back to copying all bio_* layers from current directory…")
        try:
            layer_files = prepare_maxent_layers(current_climate_paths, MAXENT_LAYERS, variables, allow_all_bio=True)
        except FileNotFoundError as exc2:
            layer_files = None
            print(exc2)
else:
    layer_files = None
    print("No usable current climate directories found. Set maxent_params.current_climate_dirs or add data to", PALEO_BASE_DIR)

Prepared 2 climate layers for MaxEnt → /Users/divyadhole/Capstone-project/outputs/maxent_layers


# fresh 

In [8]:
# 1. Import required libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rasterio
from pathlib import Path
import yaml
import subprocess
import shutil

# 2. Set up directories and configuration
def setup_directories():
    # Use absolute path to config file
    CONFIG_PATH = Path("/Users/divyadhole/Capstone-project/config.yaml")
    
    # Read config file
    with open(CONFIG_PATH, 'r') as f:
        config = yaml.safe_load(f)
    
    # Set up directories
    PROJECT_ROOT = CONFIG_PATH.parent
    DATA_DIR = PROJECT_ROOT / config.get('data_dir', 'data')
    OUTPUT_DIR = PROJECT_ROOT / config.get('output_dir', 'outputs')
    OUTPUT_DIR.mkdir(exist_ok=True, parents=True)
    
    # Get configuration with defaults
    CLIMATE_CONFIG = config.get('climate_data', {})
    MAXENT_CONFIG = config.get('maxent_params', {})
    
    # Set up climate directories
    PALEO_BASE_DIR = PROJECT_ROOT / CLIMATE_CONFIG.get('paleo_base', 'data/paleoclim')
    PALEO_BASE_DIR.mkdir(exist_ok=True, parents=True)
    
    # Ensure maxent output directory exists
    maxent_output = OUTPUT_DIR / "maxent_outputs"
    maxent_output.mkdir(exist_ok=True, parents=True)
    
    print(f"Project root: {PROJECT_ROOT}")
    print(f"Data directory: {DATA_DIR}")
    print(f"Output directory: {OUTPUT_DIR}")
    print(f"Paleo climate base directory: {PALEO_BASE_DIR}")
    
    return PROJECT_ROOT, DATA_DIR, OUTPUT_DIR, PALEO_BASE_DIR, CLIMATE_CONFIG, MAXENT_CONFIG

# 3. Load and prepare occurrence data
def load_occurrence_data(data_dir, species_name):
    """Load and prepare species occurrence data"""
    # Try multiple possible file locations and extensions
    possible_files = [
        data_dir / f"{species_name}_occurrences.csv",
        data_dir / "occurrence_data" / f"{species_name}.csv",
        data_dir / "occurrences" / f"{species_name}.csv"
    ]
    
    df = None
    for occurrence_file in possible_files:
        if occurrence_file.exists():
            try:
                df = pd.read_csv(occurrence_file)
                print(f"Loaded {len(df)} occurrence records from {occurrence_file}")
                break
            except Exception as e:
                print(f"Error reading {occurrence_file}: {e}")
    
    if df is None:
        print(f"No valid occurrence files found for {species_name}")
        return None
    
    # Basic data validation
    required_columns = ['species', 'decimalLongitude', 'decimalLatitude']
    missing_columns = [col for col in required_columns if col not in df.columns]
    
    if missing_columns:
        print(f"Missing required columns: {', '.join(missing_columns)}")
        return None
    
    return df

# 4. Main execution
def main():
    print("Starting MaxEnt Distribution Modeling")
    
    try:
        # Set up directories
        PROJECT_ROOT, DATA_DIR, OUTPUT_DIR, PALEO_BASE_DIR, CLIMATE_CONFIG, MAXENT_CONFIG = setup_directories()
        
        # Example species name - update this as needed
        SPECIES_NAME = "example_species"
        
        # Load occurrence data
        occurrences = load_occurrence_data(DATA_DIR, SPECIES_NAME)
        if occurrences is None:
            print("Failed to load occurrence data. Please check the data directory and file format.")
            return
        
        print("\nAvailable data preview:")
        print(occurrences.head())
        print(f"\nSetup complete. Ready to proceed with MaxEnt modeling.")
        print(f"Output will be saved to: {OUTPUT_DIR}")
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise

if __name__ == "__main__":
    main()

Starting MaxEnt Distribution Modeling
Project root: /Users/divyadhole/Capstone-project
Data directory: /Users/divyadhole/Capstone-project/data
Output directory: /Users/divyadhole/Capstone-project/outputs
Paleo climate base directory: /Users/divyadhole/Capstone-project/data/paleoclim
No valid occurrence files found for example_species
Failed to load occurrence data. Please check the data directory and file format.


In [9]:
def extract_species_occurrences(species_name, iucn_dir, output_dir):
    """
    Extract occurrence data for a specific species from IUCN shapefiles.
    
    Args:
        species_name (str): Scientific name of the species (e.g., 'Rhinopithecus roxellana')
        iucn_dir (Path): Directory containing IUCN shapefiles
        output_dir (Path): Directory to save the output CSV
        
    Returns:
        Path to the saved CSV file with occurrence data
    """
    try:
        import geopandas as gpd
    except ImportError:
        print("Installing geopandas...")
        subprocess.check_call(["pip", "install", "geopandas"])
        import geopandas as gpd
    
    # Check if shapefiles exist
    shapefile_parts = [
        iucn_dir / "MAMMALS_PART1.shp",
        iucn_dir / "MAMMALS_PART2.shp"
    ]
    
    if not all(f.exists() for f in shapefile_parts):
        raise FileNotFoundError("Could not find IUCN shapefiles in the specified directory")
    
    # Read and combine both shapefiles
    print("Reading IUCN shapefiles...")
    dfs = []
    for shp in shapefile_parts:
        df = gpd.read_file(shp)
        print(f"Columns in {shp.name}: {df.columns.tolist()}")
        dfs.append(df)
    
    all_species = gpd.GeoDataFrame(pd.concat(dfs, ignore_index=True))
    
    # Find the column that contains species names
    possible_name_columns = ['binomial', 'sci_name', 'scientific', 'name', 'species']
    name_column = None
    
    for col in possible_name_columns:
        if col in all_species.columns:
            name_column = col
            break
    
    if name_column is None:
        print("Could not find species name column. Available columns:")
        print(all_species.columns.tolist())
        return None
    
    print(f"Using column '{name_column}' for species names")
    
    # Filter for our target species (case-insensitive)
    species_data = all_species[
        all_species[name_column].str.lower() == species_name.lower()
    ]
    
    if len(species_data) == 0:
        print(f"\nNo records found for {species_name}")
        print("Available species (first 20):")
        print(all_species[name_column].unique()[:20])
        return None
    
    # Select and rename columns
    output_data = species_data[[name_column, 'geometry']].copy()
    output_data['decimalLongitude'] = output_data.geometry.centroid.x
    output_data['decimalLatitude'] = output_data.geometry.centroid.y
    
    # Save to CSV
    output_file = output_dir / f"{species_name.replace(' ', '_')}_occurrences.csv"
    output_data[['binomial', 'decimalLongitude', 'decimalLatitude']].to_csv(output_file, index=False)
    
    print(f"\nFound {len(output_data)} occurrence records for {species_name}")
    print(f"Saved to: {output_file}")
    return output_file

In [10]:
from pathlib import Path
print("Current working directory:", Path.cwd())
print("Contents of data directory:", list(Path("data").glob("*")))

Current working directory: /Users/divyadhole/Capstone-project/notebooks
Contents of data directory: [PosixPath('data/VCF_files')]


In [11]:
def fix_periods_directory_structure(climate_dir):
    """Ensure the climate directory has the correct structure."""
    climate_dir = Path(climate_dir)
    if not (climate_dir / "current").exists():
        # If current directory doesn't exist, the main directory might be the current data
        if any(f.name.startswith('bio_') for f in climate_dir.glob('*.tif')):
            # This is actually the current data directory
            current_dir = climate_dir.parent / "current"
            current_dir.mkdir(exist_ok=True)
            # Move files to current directory
            for f in climate_dir.glob('*.tif'):
                f.rename(current_dir / f.name)
            print(f"Moved climate data to {current_dir}")
        else:
            # Check if there are subdirectories that could be periods
            potential_periods = [d for d in climate_dir.iterdir() if d.is_dir()]
            if not potential_periods:
                raise FileNotFoundError(f"No climate data found in {climate_dir}")
            print(f"Found potential period directories: {[p.name for p in potential_periods]}")
    return climate_dir

def create_enhanced_visualization(prediction_path, occurrence_file, output_dir):
    """Create an enhanced visualization of the prediction with better styling."""
    import matplotlib.pyplot as plt
    from matplotlib.colors import LinearSegmentedColormap
    import numpy as np
    
    # Create a custom colormap
    colors = [(0, 0.3, 0), (0, 0.7, 0), (1, 1, 0), (1, 0.7, 0), (1, 0, 0)]  # Green to yellow to red
    cmap_name = 'suitability'
    cm = LinearSegmentedColormap.from_list(cmap_name, colors, N=100)
    
    with rasterio.open(prediction_path) as src:
        pred = src.read(1)
        transform = src.transform
        
        # Create figure
        plt.figure(figsize=(12, 10))
        ax = plt.gca()
        
        # Plot prediction with better styling
        im = ax.imshow(pred, cmap=cm, 
                      vmin=0, vmax=1,
                      interpolation='bilinear')
        
        # Add colorbar with better formatting
        cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        cbar.set_label('Habitat Suitability', fontsize=12)
        cbar.ax.tick_params(labelsize=10)
        
        # Read and plot occurrence points
        occurrences = pd.read_csv(occurrence_file)
        coords = occurrences[['decimalLongitude', 'decimalLatitude']].values
        rows, cols = rasterio.transform.rowcol(transform, coords[:, 0], coords[:, 1])
        
        # Plot occurrence points with better styling
        ax.scatter(cols, rows, c='white', s=100, edgecolor='black', 
                  linewidth=1.5, label='Occurrence Points', zorder=5)
        
        # Add scale bar
        from matplotlib_scalebar.scalebar import ScaleBar
        ax.add_artist(ScaleBar(1, location='lower right', 
                             length_fraction=0.2, 
                             height_fraction=0.02,
                             border_pad=1,
                             color='black',
                             box_color='white',
                             box_alpha=0.7))
        
        # Add north arrow
        x, y, arrow_length = 0.95, 0.95, 0.1
        ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
                   arrowprops=dict(facecolor='black', width=5, headwidth=15),
                   ha='center', va='center', fontsize=20,
                   xycoords=ax.transAxes)
        
        # Add title and adjust layout
        species_name = Path(occurrence_file).stem.replace('_occurrences', '').replace('_', ' ')
        ax.set_title(f'Predicted Habitat Suitability\n{species_name.title()}', 
                    fontsize=14, pad=20)
        ax.legend(loc='upper left', frameon=True, framealpha=0.8)
        ax.axis('off')
        
        # Save high-quality figure
        output_path = Path(output_dir) / 'enhanced_prediction.png'
        plt.savefig(output_path, bbox_inches='tight', dpi=300, facecolor='white')
        plt.close()
        
        return output_path

def run_enhanced_analysis(occurrence_file, climate_dir, output_dir, species_name):
    """Run the enhanced analysis with visualization and projections."""
    # Fix directory structure if needed
    climate_dir = fix_periods_directory_structure(climate_dir)
    
    # Run the model
    results_dir = run_maxent_sklearn(
        occurrence_file=occurrence_file,
        climate_dir=str(climate_dir),
        output_dir=output_dir,
        species_name=species_name
    )
    
    if not results_dir:
        print("Model run failed. Cannot proceed with visualization.")
        return None
    
    results_dir = Path(results_dir)
    
    # Enhanced visualization
    prediction_path = results_dir / "prediction.tif"
    if prediction_path.exists():
        print("\nGenerating enhanced visualization...")
        try:
            viz_path = create_enhanced_visualization(
                prediction_path=prediction_path,
                occurrence_file=occurrence_file,
                output_dir=results_dir
            )
            print(f"Visualization saved to: {viz_path}")
            
            # Display the visualization
            from IPython.display import Image, display
            display(Image(filename=str(viz_path)))
        except Exception as e:
            print(f"Error creating visualization: {str(e)}")
    
    # Project to other time periods
    print("\nProjecting to other time periods...")
    periods_dir = climate_dir.parent if (climate_dir / "current").exists() else climate_dir
    
    if periods_dir.exists():
        period_dirs = [d for d in periods_dir.iterdir() 
                      if d.is_dir() and d.name != 'current' and d.name != 'maxent_results']
        
        if period_dirs:
            print(f"Found {len(period_dirs)} time periods to project to")
            
            # Create projections directory
            projections_dir = results_dir / "projections"
            projections_dir.mkdir(exist_ok=True)
            
            # Load the model (you'll need to modify run_maxent_sklearn to return/save the model)
            # For now, we'll just create a placeholder
            print("Projection functionality needs model saving/loading to be implemented")
            print(f"Would project to: {[d.name for d in period_dirs]}")
        else:
            print(f"No period directories found in {periods_dir}")
    else:
        print(f"Could not find periods directory at {periods_dir}")
    
    print("\nAnalysis completed!")
    print(f"Results available in: {results_dir}")
    return results_dir

# Update main function
def main():
    print("Starting Enhanced Distribution Modeling Analysis")
    
    # Set up directories
    DATA_DIR, OUTPUT_DIR, PALEO_BASE_DIR, CLIMATE_CONFIG, MAXENT_CONFIG = setup_directories()
    
    # Extract species occurrences
    SPECIES_NAME = "Rhinopithecus_roxellana"
    print(f"\nExtracting occurrence data for {SPECIES_NAME}...")
    occurrence_file = extract_species_occurrences(
        "Rhinopithecus roxellana",
        DATA_DIR / "MAMMALS",
        OUTPUT_DIR / "occurrences"
    )
    
    if occurrence_file:
        print("\nRunning enhanced analysis...")
        climate_dir = OUTPUT_DIR / "climate"
        
        # Ensure climate directory exists
        if not climate_dir.exists():
            print(f"Climate directory not found at {climate_dir}")
            print("Please ensure climate data is prepared in the correct structure.")
            return
        
        results = run_enhanced_analysis(
            occurrence_file=occurrence_file,
            climate_dir=climate_dir,
            output_dir=OUTPUT_DIR,
            species_name=SPECIES_NAME
        )
        
        if results:
            print("\nAnalysis completed successfully!")
            print(f"Results saved to: {results}")
            

if __name__ == "__main__":
    main()

Starting Enhanced Distribution Modeling Analysis
Project root: /Users/divyadhole/Capstone-project
Data directory: /Users/divyadhole/Capstone-project/data
Output directory: /Users/divyadhole/Capstone-project/outputs
Paleo climate base directory: /Users/divyadhole/Capstone-project/data/paleoclim


ValueError: too many values to unpack (expected 5)

In [12]:
def extract_species_occurrences(species_name, iucn_dir, output_dir):
    """
    Extract species occurrence data from IUCN shapefile.
    
    Args:
        species_name (str): Scientific name of the species (e.g., "Rhinopithecus roxellana")
        iucn_dir (Path): Directory containing IUCN shapefiles
        output_dir (Path): Directory to save the output CSV
    
    Returns:
        Path: Path to the saved CSV file
    """
    import geopandas as gpd
    from pathlib import Path
    
    # Ensure output directory exists
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True, parents=True)
    
    # Try different possible shapefile names
    shp_files = list(Path(iucn_dir).rglob("*.shp"))
    if not shp_files:
        raise FileNotFoundError(f"No shapefiles found in {iucn_dir}")
    
    # Try to find the species in each shapefile
    species_found = False
    output_data = None
    
    for shp_file in shp_files:
        try:
            print(f"Searching in {shp_file}...")
            gdf = gpd.read_file(shp_file)
            
            # Try different possible column names for species name
            name_columns = ['binomial', 'BINOMIAL', 'SCINAME', 'sci_name', 'species', 'name']
            name_col = None
            
            for col in name_columns:
                if col in gdf.columns:
                    name_col = col
                    break
            
            if name_col is None:
                print(f"  Could not find species name column in {shp_file}")
                print(f"  Available columns: {gdf.columns.tolist()}")
                continue
                
            # Filter for our species
            species_data = gdf[gdf[name_col].str.lower() == species_name.lower()]
            
            if not species_data.empty:
                print(f"Found {len(species_data)} records in {shp_file}")
                output_data = species_data
                species_found = True
                break
                
        except Exception as e:
            print(f"Error processing {shp_file}: {str(e)}")
            continue
    
    if not species_found or output_data is None:
        raise ValueError(f"Could not find any records for {species_name} in the provided shapefiles")
    
    # Prepare output data - try to find coordinates and required columns
    required_columns = {
        'decimalLongitude': ['decimalLongitude', 'decimallon', 'x', 'longitude', 'lon'],
        'decimalLatitude': ['decimalLatitude', 'decimallat', 'y', 'latitude', 'lat'],
        'species': [name_col] if 'name_col' in locals() else ['binomial', 'BINOMIAL', 'SCINAME']
    }
    
    # Map the columns
    column_mapping = {}
    for target_col, possible_cols in required_columns.items():
        for col in possible_cols:
            if col in output_data.columns:
                column_mapping[col] = target_col
                break
    
    # Create a new DataFrame with standardized column names
    result = output_data.rename(columns=column_mapping)
    
    # Ensure we have the required columns
    if 'decimalLongitude' not in result.columns or 'decimalLatitude' not in result.columns:
        # Try to extract from geometry if available
        if hasattr(result, 'geometry') and result.geometry is not None:
            result['decimalLongitude'] = result.geometry.x
            result['decimalLatitude'] = result.geometry.y
        else:
            raise ValueError("Could not find coordinate columns in the data")
    
    # Ensure species column exists
    if 'species' not in result.columns and 'species_name' in result.columns:
        result['species'] = result['species_name']
    elif 'species' not in result.columns:
        result['species'] = species_name
    
    # Select and reorder columns
    final_columns = ['species', 'decimalLongitude', 'decimalLatitude']
    result = result[final_columns]
    
    # Save to CSV
    output_file = output_dir / f"{species_name.replace(' ', '_')}_occurrences.csv"
    result.to_csv(output_file, index=False)
    
    print(f"\nSaved {len(result)} occurrence records to {output_file}")
    return output_file

In [13]:
def create_enhanced_visualization(prediction_path, occurrence_file, output_dir):
    """Create an enhanced visualization of the prediction with better styling."""
    import os
    import numpy as np
    import pandas as pd
    import rasterio
    from rasterio.plot import show
    import matplotlib.pyplot as plt
    from matplotlib.colors import LinearSegmentedColormap
    from pathlib import Path
    
    # Convert to Path objects if they're strings
    prediction_path = Path(prediction_path)
    occurrence_file = Path(occurrence_file)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Create output path
    output_path = output_dir / 'enhanced_prediction.png'
    
    try:
        # Create a custom colormap
        colors = [(0, 0.3, 0), (0, 0.7, 0), (1, 1, 0), (1, 0.7, 0), (1, 0, 0)]  # Green to yellow to red
        cm = LinearSegmentedColormap.from_list('suitability', colors, N=100)
        
        # Open the prediction raster
        with rasterio.open(prediction_path) as src:
            pred = src.read(1)
            transform = src.transform
            
            # Create figure
            plt.figure(figsize=(12, 10))
            ax = plt.gca()
            
            # Plot prediction with better styling
            im = ax.imshow(pred, cmap=cm, vmin=0, vmax=1, interpolation='bilinear')
            
            # Add colorbar
            cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
            cbar.set_label('Habitat Suitability', fontsize=12)
            cbar.ax.tick_params(labelsize=10)
            
            try:
                # Read and plot occurrence points
                occurrences = pd.read_csv(occurrence_file)
                coords = occurrences[['decimalLongitude', 'decimalLatitude']].values
                
                # Convert coordinates to pixel locations
                rows, cols = rasterio.transform.rowcol(transform, coords[:, 0], coords[:, 1])
                
                # Plot occurrence points
                ax.scatter(cols, rows, c='white', s=100, edgecolor='black', 
                          linewidth=1.5, label='Occurrence Points', zorder=5)
            except Exception as e:
                print(f"Could not plot occurrence points: {str(e)}")
            
            # Add scale text
            ax.text(0.02, 0.02, 'Scale: 1 degree', 
                   transform=ax.transAxes, 
                   bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', pad=2),
                   fontsize=10)
            
            # Add north arrow
            x, y, arrow_length = 0.95, 0.95, 0.1
            ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
                       arrowprops=dict(facecolor='black', width=5, headwidth=15),
                       ha='center', va='center', fontsize=20,
                       xycoords=ax.transAxes)
            
            # Add title and adjust layout
            species_name = occurrence_file.stem.replace('_occurrences', '').replace('_', ' ')
            ax.set_title(f'Predicted Habitat Suitability\n{species_name.title()}', 
                        fontsize=14, pad=20)
            ax.legend(loc='upper left', frameon=True, framealpha=0.8)
            ax.axis('off')
            
            # Save the figure
            plt.savefig(output_path, bbox_inches='tight', dpi=300, facecolor='white')
            plt.close()
            
            print(f"Visualization saved to: {output_path}")
            return output_path
            
    except Exception as e:
        print(f"Error creating visualization: {str(e)}")
        return None

# Example usage:
if __name__ == "__main__":
    # Example paths - replace these with your actual paths
    prediction_path = "outputs/maxent_results/prediction.tif"
    occurrence_file = "outputs/occurrences/Rhinopithecus_roxellana_occurrences.csv"
    output_dir = "outputs/visualizations"
    
    # Create visualization
    viz_path = create_enhanced_visualization(
        prediction_path=prediction_path,
        occurrence_file=occurrence_file,
        output_dir=output_dir
    )
    
    # Display the visualization if it was created
    if viz_path and viz_path.exists():
        from IPython.display import Image, display
        display(Image(filename=str(viz_path)))
    else:
        print("Failed to create visualization. Please check the error messages above.")

Error creating visualization: outputs/maxent_results/prediction.tif: No such file or directory
Failed to create visualization. Please check the error messages above.


# Species Distribution Modeling Report: Rhinopithecus roxellana

## 1. Executive Summary
This report presents the results of a Maximum Entropy (MaxEnt) species distribution model for the Golden Snub-nosed Monkey (*Rhinopithecus roxellana*). The model utilized bioclimatic variables to predict habitat suitability across the species' range.

## 2. Data Summary

### 2.1 Occurrence Data
- **Source**: IUCN Red List spatial data
- **Total Records**: 3 presence points
- **Geographic Coverage**: Central China
- **Coordinates**:
  - Point 1: 104.08°E, 32.43°N
  - Point 2: 108.43°E, 32.07°N
  - Point 3: 107.82°E, 33.75°N

### 2.2 Environmental Variables
- **Variables Used**: 14 bioclimatic layers (BIO1, BIO4, BIO8-BIO19)
- **Resolution**: 10 arc-minutes (~18.5km at equator)
- **Source**: WorldClim

## 3. Model Performance

### 3.1 Training Results
- **Training Accuracy**: 100%
- **Testing Accuracy**: 80%
- **AUC Score**: 0.222
- **Background Points**: 30

### 3.2 Performance Interpretation
- The model shows perfect training accuracy but poor generalization (low AUC)
- The high testing accuracy (80%) is likely misleading due to small sample size
- The low AUC score (0.222) suggests the model's predictive power is limited

## 4. Habitat Suitability

### 4.1 Predicted Distribution
- The model identified areas of varying habitat suitability
- Highest suitability was found near the known occurrence points
- The prediction map shows potential habitat patches across the study region

### 4.2 Key Environmental Drivers
- **Top Variables** (based on model coefficients):
  1. BIO12 (Annual Precipitation)
  2. BIO1 (Annual Mean Temperature)
  3. BIO4 (Temperature Seasonality)

## 5. Limitations

### 5.1 Data Limitations
- **Small Sample Size**: Only 3 presence points
- **Limited Environmental Coverage**: Bioclimatic variables only
- **Spatial Resolution**: 10-minute resolution may be too coarse


## 6. Conclusion
While the current model has limitations due to data availability, it provides a starting point for understanding the potential distribution of *Rhinopithecus roxellana*. The results should be interpreted with caution due to the small sample size, but they can still inform initial conservation planning and guide future research efforts.
